<h1>PRIMERA APROXIMACIÓN AL PROBLEMA<h1>

<h2>DATOS Y PROBLEMA A ESTUDIO<h2>

Para ilustrar el ejemplo se utiliza el data set: [SURVEY LUNG CANCER](https://www.kaggle.com/datasets/mysarahmadbhat/lung-cancer) cuyas caracteristicas son: 
Total no. of attributes:16
No .of instances:284
Attribute information:

    Gender: M(male), F(female)
    Age: Age of the patient
    Smoking: YES=2 , NO=1.
    Yellow fingers: YES=2 , NO=1.
    Anxiety: YES=2 , NO=1.
    Peer_pressure: YES=2 , NO=1.
    Chronic Disease: YES=2 , NO=1.
    Fatigue: YES=2 , NO=1.
    Allergy: YES=2 , NO=1.
    Wheezing: YES=2 , NO=1.
    Alcohol: YES=2 , NO=1.
    Coughing: YES=2 , NO=1.
    Shortness of Breath: YES=2 , NO=1.
    Swallowing Difficulty: YES=2 , NO=1.
    Chest pain: YES=2 , NO=1.
    Lung Cancer: YES , NO.

El problema de estudio que utilizaremos como ejemplo de aplicación es un análisis sencillo, queremos contar cuántos casos de cáncer existen en función de si los individuos son fumadores o no. 


<h2> HERRAMIENTAS UTILIZADAS <h2>

La salida es un documento html exportado desde una libreta de Jupyter Notebook. El código está escrito en lenguaje Python3 y la matemática con comandos de $\LaTeX$. 

<h2>CIFRADO HOMOMORFICO<h2>

El uso de un sistema de cifrado homomorfico está motivado por la obtención de un algoritmo de descifrazo que defina, para una clave privada, un homomorfismo de grupos entre el espacio del mensaje cifrado ($\mathcal{C}$) y el del mensaje sin cifrar ($\mathcal{M}$): 

$$ f: \mathcal{C} \rightarrow \mathcal{M} $$

asi podremos enviar la información cifrada al servicio Cloud, tratarla estando cifrada y posteriormente traer al local las conclusiones para ser descifradas. Para una primera aproximación tomamos el sistema RSA simplificado. 

La clave publica es un par $(N,e)$ donde $N = p \cdot q$ es producto de dos primos $p$ y $q$ suficientemente grandes y distintos. Por otro lado, $e$ es el llamado exponente de cifrado que es un entero positivo tal que mcd $(e,\phi(N)) = 1$, donde $\phi(N) = (p-1)(q-1)$ es la función $\phi$ de Euler. 

La clave privada, es el par $(N,d)$ donde $d$ es el exponente de descifrado (i.e. el inverso de $e$ módulo $\phi(n)$). De este modo se verifica que 

$$ e \cdot d \equiv 1\, (mod\, (p-1)(q-1))$$

así 

$$ \mathcal{M} = \mathcal{C} = (\mathbb{Z}_N)^* $$

que es el grupo multiplicativo de las unidades módulo n, cuyos elementos son 

$$ m \in \mathbb{Z}: 1 \leq m < N \wedge mcd\, (m,n) = 1 $$

quedando el homomorfismo de grupos 

$$ E: (\mathbb{Z}_N)^* \longrightarrow (\mathbb{Z}_N)^* $$
$$ m \mapsto m^e\, mod \, N $$

que es un isomorfismo, que permite descifrar un mensaje cifrado $c$ através de su inverso:  

$$D(c) = c^d \, mod \, N$$

La utilidad de esto radica en que $((\mathbb{Z}_N)^*,\; \cdot)$ es un grupo multiplicativo, por tanto 

$$E(m_1 \cdot m_2) = E(m_1) \cdot E(m_2)$$  

<h2>APLICACIÓN AL DATASET DEL CANCER DE PULMÓN<h2>

En primer lugar vamos a definir dos funciones, una para encriptar un tipo int y otra para desencriptarlo. Los parámetros de entrada para la función que encripta serán 
<ul>
<li>publicKey -> Lista que contiene: N, e </li>
<li>mensaje -> int </li>
</ul>
mientras que para la función que desencipta tendremos
<ul>
<li>privateKey -> Lista que contiene: N, d </li>
<li>criptotexto -> int </li>
</ul>

Las funciones se muestran a continuación: 

In [2]:
def E_RSA(publicKey, mensaje): 
    n = publicKey[0]
    e = publicKey[1]
    return pow(mensaje, e, n)

def D_RSA(privateKey, criptotexto): 
    n = privateKey[0]
    d = privateKey[1]
    return pow(criptotexto, d, n)

Suponemos ahora que tenemos cifrada la columna *SMOKING* del dataset. Una primera opción que se ha trabajado es exportar el csv a un data frame, pero el tipo int64 con el que trabaja pandas colisiona con el uso del método *pow*. De momento optaré por una solución más rudimentaria para encriptar la columna y poder ilustrar este ejemplo.

In [3]:
fumar = []
with open('survey_lung_cancer.csv', 'r') as archivo: 
    lineas = archivo.read().splitlines()
    #borro encabezado
    lineas.pop(0)
    for l in lineas: 
        linea = l.split(',')
        fumar.append(int(linea[2]))

De esta forma hemos creado la columna *fumar* que contiene los enteros 1 si la persona no era fumadora y 2 si la persona era fumadora. Ambas entrada de tipo int. De modo que ya podemos encriptar secuencialmente la columna. 

Para ello definimos los valores que gobiernan el ejemplo: 

In [4]:
e = 2**16 + 1                       #exponente de cifrado 
p = 409                             #primo 
q = 317                             #primo
n = p * q                           #producto de dos primos distintos 

phi_n = (p-1)*(q-1)                 #cálculo de la función phi de Euler para n

pubKey = [n, e]                     #Clave publica 
priKey = [n, pow(e,-1,phi_n)]       #clave privada

Ciframos la columna *fumar*

In [5]:
fumar_encrip = []
for elemento in fumar: 
    fumar_encrip.append(E_RSA(pubKey, elemento))

Ahora vamos hacer el cálculo pedido: encontrar cuántos pacientes de cancer de pulmón eran fumadores y cuántos no. En primer lugar, las entradas del dataset son el 1 y un número primero. Por el teorema fundamental de la aritmética sabemos que todo entero positivo mayor que 1 es un número primo o bien un único producto de números primos. De modo que el número de pacientes fumadores se puede obtener facilmente, ya que conocemos los factores involucrados en la descomposición del producto de todos los elementos de la columna *fumar_encrip* por lo que sólo debemos conocer sus exponentes para extraer la información pedida. 

Calculamos el productorio módulo n de la *fumar_encrip*

In [6]:
from math import prod
producto = (prod(fumar_encrip)) % n
producto

84172

Esta operación con los datos encriptados se habría hecho en la estructura del proveedor cloud y exportariamos al local, unicamente, la variable *producto*. 

Una vez en local, el usuario conoceria la clave privada por lo que podría desencriptarlo y *factorizarlo* para extraer la información buscada. 

En este caso, por ser una de las entradas un 1, el único primo es 2, por lo que buscamos su exponente y mediante la diferencia respecto al total obtenemos el número de pacientes no fumadores. La función que factorizará *producto* simplemente eleva iterativamente 2, una unidad más en cada paso, hasta obtener el valor de la variable *producto*.

In [7]:
def factP(mensaje_descifrado, n): 
    result = 0
    num = 0
    while(num != mensaje_descifrado): 
        result += 1
        num = pow(2,result,n)
        
    return result 

In [8]:
fumadores = factP(D_RSA(priKey, producto), n)
total = len(fumar)
no_fumadores = abs(total - fumadores)
print(f'El total de pacientes fumadores es: {fumadores}. \nEl total de pacientes no fumadores es: {no_fumadores}. \nDe un total de {total} pacientes.')

El total de pacientes fumadores es: 174. 
El total de pacientes no fumadores es: 135. 
De un total de 309 pacientes.


<h2>LIMITACIONES<h3>

El proceso que se acaba de describir es inútil desde el punto de vista de la seguridad. Al ser un proceso determinista, es decir, los pacientes fumadores y no fumadores se cifran siempre de la misma manera, no podemos presuponer que alguien que quiera hacer un uso fraudulento de la información desconozca los elementos de $(\mathbb{Z}_n)^*$ que se han utilizado para representar las condiciones de ser o no fumador (*principio de Kerckhoffs*). 

Esto se puede traducir en que, en el peor caso, con un par evaluaciones obtenemos si un paciente es o no fumador: 

In [10]:
valor = E_RSA(pubKey, 1)

if (not E_RSA(pubKey, 1) == valor):
    E_RSA(pubKey, 2) == valor
    print("Representa: Es fumador")
else: 
    print("Representa: No es fumador")

Representa: No es fumador


La probabiliad de distinguir al fumador del no fumador es 1. El objetivo es que no ser capaces de acertar con probabilidad significativamente mayor que $\dfrac{1}{2}$. La definición asintótica de la probabiliad insignificante supone que, si alguién que intenta distinguir los mensajes, lo hace con probabiliad $\dfrac{1}{2}+\dfrac{1}{f(k)}$, donde $k$ es la longitud del módulo RSA y $f(k)$ una función polinómica, entonces el algoritmo no es seguro. 

<h1>MEJORANDO EL SISTEMA<h1>

Un algoritmo de cifrado que es homomórfico y CPA seguro (se dice que un procedimiento es CPA seguro si en tiempo polinómico un atacante no puede distinguir entre dos criptotextos partiendo de dos textos sin cifrar elegidos por él) es el de Paillier.    

Las fases que gobiernan el algoritmo de Paillier son tres, a saber: 
* Generar las claves. Se generan dos primos aleatorios $p$ y $q$ y se obtiene la clave pública $n=pq$ y la clave privada $(n, \phi(n))$. Por simplicidad supondremos que $p$ y $q$ tienen la misma longitud. 
* Cifrar. El conjunto de textos sin cifrar será $\mathbb{Z}_n$ y el de ciptotextos $(\mathbb{Z}_{n^2})^*$. Para cifrar un mensaje $m \in \mathbb{Z}_n$, se elige $r \in (\mathbb{Z}_{n^2})^*$ al *azar* y se calcula el criptotexto como 

$$c = E(n, m) := ((1+n)^m \cdot r^n)\, mod \, n^2 \in (\mathbb{Z}_{n^2})^*$$

* Descifrar. Para descrifrar $c$ con la clave privada $(n, \phi(n))$ se hace $$m = D((n, \phi(n)), c) := \dfrac{(c^{\phi(n)}\, mod\, n^2)-1}{n} \cdot \phi(n)^{-1} \, mod \, n$$ donde $((c^{\phi(n)}\, mod\, n^2)-1)/n$ se calcula en $\mathbb{Z}$.

Ahora bien, $\mathbb{Z}_n$ es aditivo, y $(\mathbb{Z}_{n^2})^*$ multiplicativo, por lo que la función de descifrado de Paillier lleva un producto de criptotextos en $(\mathbb{Z}_{n^2})^*$ la suma de los textos sin cifrar en $\mathbb{Z}_{n}$. Sean $m_1$ y $m_2 \in \mathbb{Z}_n$, entonces $c_1 = E(n, m_1)$ y $c_2 = E(n, m_2)$ de donde  

$$D((n, \phi(n), c), (c_1 \cdot c_2)\, mod \, n^2) = (m_1 + m_2) \, mod \, n$$ 

La aplicación $D: (\mathbb{Z}_{n^2})^* \rightarrow \mathbb{Z}_{n}$ inducida por el algoritmo de descifrado con una clave fija es un homomorfismo de grupos (no se puede afirmar lo mismo de del cifrado por ser probabilista). No obstante, es cierto que para cifrar la suma de varios mensajes se puede tomar como criptotexto el productorio de los criptotextos asociados a cada mensaje. 

<h3>BIBLIOGRAFIA<h3>

Milanov, E. (2009). The RSA algorithm. RSA laboratories, 1-11.

Pardo, J. L. G. (2012). Cifrado homomórfico: ejemplos y aplicaciones. Gaceta de la Real Sociedad Matematica Española, 15(4), 697-712.

